<개요>
1. 데이터 로드
2. 레시피 s

### Data Loading


- 오라클에서 가져오는 방법 추가

In [6]:
import pandas as pd
data = pd.read_csv(r'data/RecipeData.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/RecipeData.csv'

### Doc2Vec으로 임베딩하기

- 전처리 없이 샘플 10000개 임베딩
- 1. recipe_step을 형태소 분석기로 전처리
- 2. gensim doc2vec으로 유사도 분석

In [3]:
import random 

random.seed(77)
toy = data[["recipe_title", "recipe_steps"]].sample(10000)

NameError: name 'data' is not defined

In [60]:
toy

,recipe_title,recipe_ingredients
107774,닭백숙 끓이는법 면역력을 높여라,"{'재료': ['영계닭1마리', '찹쌀180g', '대추1줌', '통마늘9알정도',..."
153011,엄마표 간식 초코꽃쿠키 만들기,"{'재료': ['박력분90g', '슈가파우더55g', '코코아파우더15g', '버터..."
137584,상큼하게 즐기는 스팸 야채 무쌈말이,"{'재료': ['스팸', '쌈용무', '팽이버섯', '파프리카', '무순'], '양..."
83478,홈메이드 맛술 만들기...생강 활용팁!!,"{'재료': ['생강1컵', '매실액1/2컵', '찹쌀밥2/3컵', '청주1병']}"
29760,두반장으로 만드는 마파두부,"{'재료': ['돼지고기250g', '두부1모', '양파1개', '청양고추3개', ..."
...,...,...
124398,#초스피드 다이어트음식 해초샐러드만들기 #각종 채소들을 추가해서 만드는 상큼하고 바...,"{'재료': ['해초샐러드1팩', '빨간파프리카1/2개', '노랑파프리카1/2개',..."
112108,태국식 커리 스파게티,"{'재료': ['페튜치네100g', '닭안심70g', '바질20g', '마늘슬라이스..."
39690,돼지목살 양념구이,"{'재료': ['돼지목살600g', '대파1줄', '배1/4개', '양파1/4개',..."
152244,허니레몬은 사랑의 맛!! 크림 치즈 머핀,"{'재료': ['레몬1개', '꿀2Ts', '버터80g', '설탕80g', '달걀1..."


In [37]:
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

okt = Okt()
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()

In [48]:
tagged_corpus_list = []

for index, row in tqdm(toy.iterrows(), total=len(toy)):
  text = row['recipe_ingredients']
  tag = row['recipe_title']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=okt.morphs(text)))

100%|██████████| 10000/10000 [00:41<00:00, 240.76it/s]


In [49]:

tagged_corpus_list[10]

TaggedDocument(words=["{'", '재료', "':", "['", '목살', "',", "'", '고', '체형', '카레', "',", "'", '양파', "',", "'", '당근', "',", "'", '감자', "',", "'", '호박', "',", "'", '파프리카', "',", "'", '후추', "',", "'", '소금', "']}"], tags=['집밥 백선생 목살카레스테이크'])

In [39]:
from gensim.models import doc2vec

In [50]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)

# 수정된 부분: doctags 대신 key_to_index 사용
print(f"Tag Size: {len(model.dv.key_to_index)}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=10)

# 모델 저장
model.save('recipe.doc2vec')


Tag Size: 9906 / 

In [61]:
similar_doc = model.docvecs.most_similar('닭백숙 끓이는법 면역력을 높여라')
print(similar_doc)

[('삼계탕 맛있게 만들기~!', 0.8520060777664185), ('오골계 아니닭', 0.8211135864257812), ('한방삼계탕 보양식 삼계탕 만드는법', 0.8210607767105103), ('맑은 순두부 떡만두국', 0.8117913007736206), ('한방삼계탕 여름보양식 녹두/전복/인삼 털어넣어!', 0.8084213137626648), ('비 오는 날이면 더욱 생각나는 부침개', 0.8065982460975647), ('술안주 추천 :]오징어부추부침개 로 술술 넘어가요', 0.804054319858551), ('눈개승마튀김/삼나물튀김', 0.8016625046730042), ('♥[신혼 디저트] 호두아몬드 쿠키', 0.8011251091957092), ('[비건채식] 두부전과 두부버거', 0.7969818711280823)]


C:\Users\HwijunKwon\AppData\Local\Temp\ipykernel_9512\3197389760.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar('닭백숙 끓이는법 면역력을 높여라')


In [65]:
similar_doc_2 = model.docvecs.most_similar('엄마표 간식 초코꽃쿠키 만들기')
print(similar_doc_2)

[('백설 쿠키 만들기', 0.8908436298370361), ('크림치즈타르트 만들기', 0.8726410269737244), ('조기찌개 - 딸에게# / 조기찌개만드는법', 0.8725787401199341), ('[나만의요리 - 무쇠가마솥] 완두콩밥', 0.8717539310455322), ('황금레시피 쑥전만들어 먹기', 0.8714804649353027), ('찹살을이용한꽂만들기(떡케익)', 0.8710826635360718), ('오징어 고추장찌개 - 딸에게#', 0.8694823980331421), ('물향기식 김치만두.떡만두국', 0.8694505095481873), ('계란톡! 두부 풍덩~ 두부전 레시피', 0.8686668872833252), ('생도라지 손질법 제일 쉬운방법으로 깔끔하게', 0.8679294586181641)]


C:\Users\HwijunKwon\AppData\Local\Temp\ipykernel_9512\1959178993.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc_2 = model.docvecs.most_similar('엄마표 간식 초코꽃쿠키 만들기')


### 해결해야할 과제


1. 레시피에서 하는 요리에 대한 정의가 필요함
  - 레시피 타이틀 전처리
  - 레시피 태그 전처리 
2. 레시피에 따라서 2인분, 3인분, 4인분 다양함

In [67]:
data["recipe_summary1"].value_counts()

2인분       57219
4인분       31408
1인분       25937
3인분       19687
6인분 이상    16377
5인분        3670
Name: recipe_summary1, dtype: int64